In [1]:
import numpy as np
import json
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
data_point = '0200'
npy_file_path = f'/home/colino/hawk/synthetic_ground_truth/tests/DNO-SinglePersonStaticOcc-v1-test1/RenderProduct_Replicator_62/bounding_box_3d/bounding_box_3d_{data_point}.npy'
labels_file_path = f'/home/colino/hawk/synthetic_ground_truth/tests/DNO-SinglePersonStaticOcc-v1-test1/RenderProduct_Replicator_62/bounding_box_3d/bounding_box_3d_labels_{data_point}.json'
data_npy = np.load(npy_file_path, allow_pickle=True)

In [3]:
print("Contents of the .npy file:")
print(data_npy[:])

Contents of the .npy file:
[(0,  -0.27468443, -2.0088959e-01, -9.1133174e-04,  0.28058743,  0.19025135,  1.7695086, [[ 1.23238368e-02, -9.99924123e-01,  0.00000000e+00,  0.00000000e+00], [ 9.99924123e-01,  1.23238368e-02,  0.00000000e+00,  0.00000000e+00], [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,  0.00000000e+00], [-1.27815523e+01, -2.82463748e-02, -3.76612888e-07,  1.00000000e+00]], -1.        )
 (1, -90.15808   , -4.0475250e+01, -2.6263401e+01, 30.141968  , 39.805756  , 22.65179  , [[ 8.50274507e-03, -5.26339421e-03,  9.98719756e-08,  0.00000000e+00], [ 5.26339421e-03,  8.50274507e-03, -8.66116807e-07,  0.00000000e+00], [ 3.70952819e-07,  7.89003650e-07,  9.99999978e-03,  0.00000000e+00], [-9.75380516e+00,  1.10768342e+00,  3.60265672e-01,  1.00000000e+00]],  0.74509996)
 (1, -39.84073   , -2.3655593e-06, -5.9979248e+01, 39.715454  , 14.050013  , 60.020874 , [[ 5.26339421e-03,  8.50274507e-03, -8.66335540e-07,  0.00000000e+00], [ 3.70793316e-07,  7.89286389e-07,  9.9999997

In [4]:
# Extract the first column
first_elements = [row[0] for row in data_npy]

# Count unique values in the first column
unique_values, counts = np.unique(first_elements, return_counts=True)

In [5]:
print(unique_values, counts)

[0 1] [ 1 18]


In [6]:
with open(labels_file_path, 'r') as labels_file:
    labels_data = json.load(labels_file)

In [7]:
print("Contents of the label file")
print(labels_data)

Contents of the label file
{'0': {'class': 'person_1'}, '1': {'class': 'fullpallet_1'}}


In [8]:
data_npy_full = [list(row) for row in data_npy]

In [9]:
sign_data = [row for row in data_npy_full]

In [10]:
geometries = []

In [11]:
sign_points = []
sign_transforms = []
for row in sign_data:
    x_min = row[1]
    y_min = row[2]
    x_max = row[3]
    y_max = row[4]
    z_min = row[5]
    z_max = row[6]
    
    bbox_points = [
        [x_min, y_min, z_min],
        [x_min, y_min, z_max],
        [x_min, y_max, z_min],
        [x_min, y_max, z_max],
        [x_max, y_min, z_min],
        [x_max, y_min, z_max],
        [x_max, y_max, z_min],
        [x_max, y_max, z_max]
    ]
    
    sign_points.append(bbox_points)
    
    transform = row[7]
    sign_transforms.append(transform.T)
    
    
sign_points = np.array(sign_points).astype(np.float64)
sign_transforms = np.array(sign_transforms).astype(np.float64)   
    
print(sign_points)
print(sign_transforms)

[[[-2.74684429e-01 -2.00889587e-01  1.90251350e-01]
  [-2.74684429e-01 -2.00889587e-01  1.76950860e+00]
  [-2.74684429e-01  2.80587435e-01  1.90251350e-01]
  [-2.74684429e-01  2.80587435e-01  1.76950860e+00]
  [-9.11331736e-04 -2.00889587e-01  1.90251350e-01]
  [-9.11331736e-04 -2.00889587e-01  1.76950860e+00]
  [-9.11331736e-04  2.80587435e-01  1.90251350e-01]
  [-9.11331736e-04  2.80587435e-01  1.76950860e+00]]

 [[-9.01580811e+01 -4.04752502e+01  3.98057556e+01]
  [-9.01580811e+01 -4.04752502e+01  2.26517906e+01]
  [-9.01580811e+01  3.01419678e+01  3.98057556e+01]
  [-9.01580811e+01  3.01419678e+01  2.26517906e+01]
  [-2.62634010e+01 -4.04752502e+01  3.98057556e+01]
  [-2.62634010e+01 -4.04752502e+01  2.26517906e+01]
  [-2.62634010e+01  3.01419678e+01  3.98057556e+01]
  [-2.62634010e+01  3.01419678e+01  2.26517906e+01]]

 [[-3.98407288e+01 -2.36555934e-06  1.40500126e+01]
  [-3.98407288e+01 -2.36555934e-06  6.00208740e+01]
  [-3.98407288e+01  3.97154541e+01  1.40500126e+01]
  [-3.98

In [12]:
sign_pcds = []

for i, row in enumerate(sign_points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(row)
    pcd.transform(sign_transforms[i])
    sign_pcds.append(pcd)
    
geometries += sign_pcds

In [13]:
print(len(geometries))

19


In [14]:
sign_bboxes = []

for i, pcd in enumerate(sign_pcds):
    print(f"Sign {i}")
    sign_bbox = pcd.get_oriented_bounding_box()
    sign_bbox.color = [1,0,0]
    sign_bboxes.append(sign_bbox)
    
geometries += sign_bboxes

Sign 0
Sign 1
Sign 2
Sign 3
Sign 4
Sign 5
Sign 6
Sign 7
Sign 8
Sign 9
Sign 10
Sign 11
Sign 12
Sign 13
Sign 14
Sign 15
Sign 16
Sign 17
Sign 18


In [28]:
origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1, origin=[0, 0, 0])
geometries += [origin_frame]
#geometries = sign_pcds

In [25]:
o3d.visualization.draw_geometries(geometries)